In [0]:

# NOTEBOOK 3: SILVER TO GOLD - DIMENSIONAL MODEL BUILD

# Purpose: Build Star Schema (Fact + Dimension tables) from Silver data


# : Configuration

from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

storage_account_name = "your account name"
storage_account_key = "your storage account key"

spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_key
)

silver_base_path = f"abfss://silver@{storage_account_name}.dfs.core.windows.net"
gold_base_path = f"abfss://gold@{storage_account_name}.dfs.core.windows.net"

print(" Configuration complete!")

#  Load Silver Data

print(" Loading Silver layer data...")

# Load cleaned reviews
df_reviews = spark.read.parquet(f"{silver_base_path}/reviews_cleaned")
print(f" Reviews loaded: {df_reviews.count():,} records")

# Load cleaned metadata
df_metadata = spark.read.parquet(f"{silver_base_path}/metadata_cleaned")
print(f" Metadata loaded: {df_metadata.count():,} records")


# 3: Build Dimension - dim_date


print("\n" + "="*60)
print("  Building dim_date (Date Dimension)")
print("="*60)

# Extract unique dates from reviews
df_dates = df_reviews.select(F.col("review_date").cast("date").alias("full_date")).distinct()

# Generate date attributes
df_dim_date = df_dates \
    .withColumn("date_key", F.date_format(F.col("full_date"), "yyyyMMdd").cast("int")) \
    .withColumn("year", F.year(F.col("full_date"))) \
    .withColumn("quarter", F.quarter(F.col("full_date"))) \
    .withColumn("month", F.month(F.col("full_date"))) \
    .withColumn("month_name", F.date_format(F.col("full_date"), "MMMM")) \
    .withColumn("week_of_year", F.weekofyear(F.col("full_date"))) \
    .withColumn("day_of_month", F.dayofmonth(F.col("full_date"))) \
    .withColumn("day_of_week", F.dayofweek(F.col("full_date"))) \
    .withColumn("day_name", F.date_format(F.col("full_date"), "EEEE")) \
    .withColumn("is_weekend", F.when(F.col("day_of_week").isin([1, 7]), True).otherwise(False)) \
    .orderBy("date_key")

print(f" dim_date created: {df_dim_date.count():,} unique dates")
df_dim_date.show(10)

# Write to Gold
# df_dim_date.write.mode("overwrite").parquet(f"{gold_base_path}/dim_date")

df_dim_date.write.format("delta").mode("overwrite").save(f"{gold_base_path}/dim_date")
print(" dim_date written to Gold layer")

# : Build Dimension - dim_categories


print("\n" + "="*60)
print(" Building dim_categories (Category Dimension)")
print("="*60)

# Get unique categories from metadata
df_dim_categories = df_metadata.select("main_category").distinct() \
    .withColumn("category_key", F.monotonically_increasing_id().cast("int")) \
    .withColumnRenamed("main_category", "category_name") \
    .select("category_key", "category_name")

print(f" dim_categories created: {df_dim_categories.count():,} categories")
df_dim_categories.show()

# Write to Gold
# df_dim_categories.write.mode("overwrite").parquet(f"{gold_base_path}/dim_categories")
df_dim_categories.write.format("delta").mode("overwrite").save(f"{gold_base_path}/dim_categories")

print(" dim_categories written to Gold layer")

#  Build Dimension - dim_products

print("\n" + "="*60)
print("  Building dim_products (Product Dimension)")
print("="*60)

# Use metadata as base, enrich with review aggregates
df_dim_products = df_metadata \
    .withColumn("product_key", F.monotonically_increasing_id().cast("bigint")) \
    .select(
        "product_key",
        "parent_asin",
        "title",
        "price_cleaned",
        "average_rating",
        "rating_number",
        "store",
        "main_category",
        "brand",
        "features_text",
        "description_text"
    )

print(f" dim_products created: {df_dim_products.count():,} products")
df_dim_products.show(5, truncate=True)

# Write to Gold
#df_dim_products.write.mode("overwrite").parquet(f"{gold_base_path}/dim_products")
df_dim_products.write.format("delta").mode("overwrite").save(f"{gold_base_path}/dim_products")

print(" dim_products written to Gold layer")


# 6: Build Dimension - dim_users


print("\n" + "="*60)
print(" Building dim_users (User Dimension)")
print("="*60)

# Aggregate user statistics from reviews
df_dim_users = df_reviews.groupBy("user_id").agg(
    F.min("review_date").alias("first_review_date"),
    F.count("*").alias("total_reviews"),
    F.avg("rating").alias("avg_rating_given")
) \
.withColumn("user_key", F.monotonically_increasing_id().cast("bigint")) \
.select(
    "user_key",
    "user_id",
    "first_review_date",
    "total_reviews",
    F.round("avg_rating_given", 2).alias("avg_rating_given")
)

print(f" dim_users created: {df_dim_users.count():,} users")
df_dim_users.show(10)

# Write to Gold
# df_dim_users.write.mode("overwrite").parquet(f"{gold_base_path}/dim_users")
df_dim_users.write.format("delta").mode("overwrite").save(f"{gold_base_path}/dim_users")

print(" dim_users written to Gold layer")


# : Build Fact Table - fact_reviews

print("\n" + "="*60)
print(" Building fact_reviews (Fact Table - Star Schema Core!)")
print("="*60)



dim_products = spark.read.format("delta").load(f"{gold_base_path}/dim_products")
dim_users = spark.read.format("delta").load(f"{gold_base_path}/dim_users")
dim_date = spark.read.format("delta").load(f"{gold_base_path}/dim_date")
dim_categories = spark.read.format("delta").load(f"{gold_base_path}/dim_categories")

# Prepare reviews with date_key
df_reviews_with_datekey = df_reviews \
    .withColumn("date_key", F.date_format(F.col("review_date").cast("date"), "yyyyMMdd").cast("int"))

# Join to get product_key
df_fact = df_reviews_with_datekey.join(
    dim_products.select("product_key", "parent_asin", "main_category"),
    on="parent_asin",
    how="inner"  # Inner join: only keep reviews with matching products
)

# Join to get user_key
df_fact = df_fact.join(
    dim_users.select("user_key", "user_id"),
    on="user_id",
    how="inner"
)

# Join to get category_key
df_fact = df_fact.join(
    dim_categories.select("category_key", F.col("category_name").alias("main_category")),
    on="main_category",
    how="inner"
)

# Step 2: Create fact table with measures and foreign keys
df_fact_reviews = df_fact \
    .withColumn("review_key", F.monotonically_increasing_id().cast("bigint")) \
    .select(
        "review_key",           # Primary Key
        "product_key",          # FK to dim_products
        "user_key",             # FK to dim_users
        "date_key",             # FK to dim_date
        "category_key",         # FK to dim_categories
        "rating",               # MEASURE
        "helpful_vote",         # MEASURE
        "verified_purchase",    # FACT (boolean)
        "review_text_length",   # MEASURE
        "has_images",           # FACT (boolean)
        F.col("timestamp").alias("review_timestamp")  # Keep original timestamp
    )

print(f" fact_reviews created: {df_fact_reviews.count():,} records")
df_fact_reviews.show(10)

# Data validation: check for nulls in foreign keys (should be 0!)
print("\n Data Quality Check - Null Foreign Keys:")
df_fact_reviews.select([
    F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(f"{c}_nulls")
    for c in ["product_key", "user_key", "date_key", "category_key"]
]).show()




df_fact_reviews.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("category_key") \
    .save(f"{gold_base_path}/fact_reviews")

print(" fact_reviews written to Gold layer")


# 8: Summary & Validation

print("\n" + "="*60)
print(" GOLD LAYER BUILD COMPLETE!")
print("="*60)

print("\n--Dimensional Model Summary:")
print(f" dim_date:       {df_dim_date.count():,} dates")
print(f" dim_categories: {df_dim_categories.count():,} categories")
print(f" dim_products:   {df_dim_products.count():,} products")
print(f" dim_users:      {df_dim_users.count():,} users")